## MimiBRICK Calibration Example

This notebook contain a examples of the calibration functionalities in the MimiBRICK.jl package.

### Step 1. Environment Setup

For this example, this notebook will run using the environment defined by the `Manifest.toml` and "Project.toml" files within this `examples` folder.  If you wish to follow along and type these out on your local machine, you will need to set up a matching Julia environment with the same files.

To do so, either add the packages listed in the `Project.toml` and printed below after `Pkg.status()`, or feel free to copy over the two environment files (`Project.toml` and `Manifest.toml`) onto your local machine and then activate the environment with either `Pkg.activate(".")` or `]` to enter the Pkg REPL followed by `activate .` and then a backspace to exict the Pkg REPL).

In [9]:
# Activate the examples environment 
using Pkg
Pkg.activate(".")
Pkg.status() # Check which packages are loaded

  Activating environment at `~/.julia/dev/MimiBRICK/examples/Project.toml`


      Status `~/.julia/dev/MimiBRICK/examples/Project.toml`
  [5d742f6a] CSVFiles v1.0.1
  [a93c6f00] DataFrames v1.3.4
  [91f4afc7] MimiBRICK v1.0.0-DEV `https://github.com/raddleverse/MimiBRICK.jl#master`
  [3f1801d0] MimiSNEASY v1.0.3-DEV `https://github.com/anthofflab/MimiSNEASY.jl#master`
  [8dfed614] Test


In [10]:
using MimiBRICK

### Step 2. Directories

First we define and create the directory that will hold the results. In this case we will create a folder `results` in the current `examples` directory.

In [11]:
my_output_dir = joinpath(@__DIR__, "results")

# if the path already exists, clean it out
isdir(my_output_dir) ? rm(my_output_dir, recursive = true) : nothing

# recreate the folder
mkpath(my_output_dir);

### Step 3. Run Calibration

Next we define various settings for the `MimiBRICK.run_calibration` function. Note that if you do not wish to change the default setting, then the argument can be excluded from the call. We include them below for illustrative purposes.

In [12]:
model_config            = "brick"
total_chain_length      = 1000
burnin_length           = 0
threshold_gr            = 1.1
size_subsample          = 100
calibration_start_year  = 1850
calibration_end_year    = 2017
num_walkers             = 2
start_from_priors       = false;

Note that to produce the results hosted on Zenodo, different settings were used such that the process takes over 8 hours.  Specifically, the `total_chain_length` was set to `20_000_000` (20 million) and `size_subsample` to `10_000` (ten thousand).

Next we run the calibration!

In [13]:
MimiBRICK.run_calibration(  output_dir=my_output_dir, 
                            model_config=model_config, 
                            calibration_start_year=calibration_start_year, 
                            calibration_end_year=calibration_end_year,
                            total_chain_length=total_chain_length, 
                            burnin_length=burnin_length, 
                            threshold_gr=threshold_gr, 
                            num_walkers=num_walkers,
                            size_subsample=size_subsample, 
                            start_from_priors=start_from_priors
                        );

Begin baseline calibration of brick model.



Progress:  12%|█████                                    |  ETA: 0:00:01



Progress:  24%|█████████▊                               |  ETA: 0:00:01



Progress:  31%|████████████▌                            |  ETA: 0:00:01



Progress:  46%|██████████████████▊                      |  ETA: 0:00:01


...


Progress:  75%|██████████████████████████████▊          |  ETA: 0:00:00



Progress:  84%|██████████████████████████████████▌      |  ETA: 0:00:00



Progress:  93%|██████████████████████████████████████▏  |  ETA: 0:00:00



Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
  acceptance_rate:  0.269


  1.399572 seconds (6.78 M allocations: 734.796 MiB, 18.28% gc time)
sd_glaciers  1.22
sd_greenland  2.2514
sd_antarctic  2.4503
sd_gmsl  1.0266
rho_glaciers  1.0221
rho_greenland  2.1145
rho_antarctic  1.1686
rho_gmsl  1.1214
thermal_s0  1.0256
greenland_v0  1.0412
glaciers_v0  1.1978
glaciers_s0  1.2321
antarctic_s0  1.1782
thermal_alpha  1.1662
greenland_a  1.4581
greenland_b  1.7487
greenland_alpha  1.4281
greenland_beta  1.1239
glaciers_beta0  1.7041
glaciers_n  1.4952
anto_alpha  1.3233
anto_beta  1.4826
antarctic_gamma  1.3604
antarctic_alpha  1.907
antarctic_mu  1.8297
antarctic_nu  1.6185
antarctic_precip0  1.2997
antarctic_kappa  1.6569
antarctic_flow0  1.3208
antarctic_runoff_height0  1.092
antarctic_c  1.0042
antarctic_bed_height0  1.1475
antarctic_slope  1.0109
antarctic_lambda  1.0447
antarctic_temp_threshold  1.4784
Saving calibrated parameters for brick.




The `run_calibration` function will create a subfolder for the `model_config` in the user-defined `output_dir` with a host of output files. 

### Step 4. Run Hindcast and Projections

First we run the hindcast period 1850-2017 using the `MimiBRICK.run_hindcast` function, for `model_config = brick`. For the hindcast, no RCP scenario needs to be specified, because all of them follow historical radiative forcing/emissions trends up to 2005.

The standard set of parameters that are being used for the hindcast and projection simulations are the sub-sample of 10,000 from the MCMC calibration described above (`parameters_subsample_(model_config).csv`). If you have a different parameter file that you want to run the hindcasts under, you will want to modify the section of `run_hindcast.jl` titled `Set paths for results files` (line 41).

This script will add model configuration-specific directory that was constructed above (or came with the model codes). It will create a sub-directory called `hindcast_csv` which will be populated with CSV files that include the simulated hindcasts of the model output fields. Each of these names is appended with `model_config` (`brick`, `doeclimbrick`, or `sneasybrick`) and contains one hindcast simulation for each of the sets of parameters in the sub-sample for analysis. Rows correspond to different years (1850-2017 be default) and columns each correspond to different ensemble members.

In [14]:
MimiBRICK.run_hindcast(output_dir=my_output_dir, model_config=model_config);

Next we run the projections for the period 1850-2300 (but can be modified to any period between 1765 and 2300) by using the `MimiBRICK.run_projections` function, using `model_config=brick`, `doeclimbrick` or `sneasybrick` and `rcp_scenario="RCP26"`, `"RCP45"`, `"RCP60"`, or `"RCP85"`. Note that the RCP scenario forcing files are all the same until 2005, and the provided stand-alone BRICK temperature and ocean heat forcing files cover the period 1850-2300.

This script will add to the model configuration-specific directory that was constructed above (or came with the model codes). It will create a sub-directory called `projections_csv`, and a sub-directory within there that is specific to each RCP scenario used will be created. The projections files are analogous to the hindcast files that are generated, and will populate the `projections_csv/[RCP scenario]` directory.

In [15]:
MimiBRICK.run_projections(output_dir=my_output_dir, model_config=model_config);


### Step 5. Downscale

Finally we use the `MimiBRICK.downscale_brick` function to downscale the BRICK global sea level projections to local. This uses the sea-level "fingerprints" of [Slangen et al. (2014)](https://link.springer.com/article/10.1007/s10584-014-1080-9). The downscaling routine will automatically create a subdirectory in the output directory called `localslr`. In this subdirectory, the routine will save an output file with the downscaled local mean sea level change model output.

In [17]:
# Lat and Lon for New York City
lat=40.7128 # deg N
lon=360-74.0060; # 74.0060 deg W

In [19]:
# downscale hindcast
years, lsl_hind_ens=MimiBRICK.downscale_brick(lon=lon, lat=lat, results_dir=my_output_dir, proj_or_hind="hind", ensemble_or_map="ensemble", model_config="brick", rcp_scenario="RCP85");

# downscale proejction
years, lsl_proj_ens=MimiBRICK.downscale_brick(lon=lon, lat=lat, results_dir=my_output_dir, proj_or_hind="proj", ensemble_or_map="ensemble", model_config="brick", rcp_scenario="RCP85");

#### 